In [62]:
# Import Dependencies
import pandas as pd
import gmaps
import requests
from config import g_key
from geopy import distance
from geopy import Nominatim
import numpy as np

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [18]:
# Get Address from User
street = input("Enter Address Line 1 (Street Address): ").strip()
city = input("Enter City: ").strip()
state = input("Enter State: ")
zip_code = str(input("Enter Zip/Postal Code: ")).strip()

Enter Address Line 1 (Street Address): 506 E. Locust St.
Enter City: San Antonio
Enter State: Texas
Enter Zip/Postal Code: 78212
 You entered: 506 E. Locust St., San Antonio, Texas, 78212. Is this correct? (Yes or No)


In [28]:
# Combines individual address inputs into one variable
address = street + ", " + city + ", " + state + ", " + zip_code

# Asks if address is correct
print(f" You entered: {address}. \n Is this correct?")
ans = input("Yes or No: ").lower()

 You entered: 506 E. Locust St., San Antonio, Texas, 78212. 
 Is this correct?
Yes or No: NO
no


In [32]:
# Returns lat/long from address
geolocator = Nominatim(user_agent="my_user_agent")
location = geolocator.geocode(address)
long_address = location
lat = location.latitude
long = location.longitude
print(long_address)
print(lat, long)

506, East Locust Street, Tobin Hill, San Antonio, Bexar County, Texas, 78212, United States
29.44568042857143 -98.48886314285714


In [ ]:
# State abreviations; EPA API uses abbreviations, address has whole name
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [38]:
# Returns abbreviation of state for URL
url_state = us_state_abbrev[state]

In [49]:
# EPA API Base URL for Active Superfund Sites
# Returns all location within the state and puts into Pandas DF

request = requests.get(f'https://data.epa.gov/efservice/SEMS_ACTIVE_SITES/SITE_STATE/CONTAINING/{url_state}/JSON').json()

In [65]:
sf_sites_all = pd.DataFrame(request)
sf_sites_all

,RG,SITE_ID,SITE_EPA_ID,SITE_NAME,SITE_STRT_ADRS1,SITE_STRT_ADRS2,SITE_CITY_NAME,SITE_STATE,SITE_ZIP_CODE,SITE_CONG_DISTRICT,SITE_CNTY_NAME,SITE_FIPS_CODE,LATITUDE,LONGITUDE,ALT_AGREEMENT,NPL,FF,NON_NPL_STATUS
0,6,0605181,TX0000605181,ROCKWALL MERCURY,3909 MEDITERRANEAN,None,ROCKWALL,TX,None,None,ROCKWALL,48397.0,NaN,NaN,N,Not on the NPL,N,N
1,6,0605258,TX0000605258,GULF NUCLEAR,2717 WEST 81ST STREET,None,ODESSA,TX,79764,None,ECTOR,48135.0,NaN,NaN,N,Not on the NPL,N,N
2,6,0605264,TX0000605264,BRINE SERVICE COMPANY,NORTHEAST OF INTERSECTION AT IH-37 AND,GOLDSTON ROAD,CORPUS CHRISTI,TX,78409,27,NUECES,48355.0,27.815372,-97.508606,N,Currently on the Final NPL,N,N
3,6,0605265,TX0000605265,URBAN MACHINE,8236 TRAVELAIR,None,HOUSTON,TX,77061,None,HARRIS,48201.0,NaN,NaN,N,Not on the NPL,N,N
4,6,0605291,TX0000605291,LITTLE BIT RAD SITE,CORNER OF FORSYTHE AND SUNNYSIDE,None,BEAUMONT,TX,77707,None,JEFFERSON,48245.0,NaN,NaN,N,Not on the NPL,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,6,0607102,TXR988077640,"BATTERY RECLAMATION, INC.",2110 WESTERN AVE.,None,PECOS,TX,79772,None,REEVES,48389.0,NaN,NaN,N,Not on the NPL,N,N
450,6,0605147,TXSFN0605147,MADISONVILLE TIRE/DRUM FIRE,3 MILES SE OF MADISONVILLE,None,MADISONVILLE,TX,77864,None,MADISON,48313.0,NaN,NaN,N,Not on the NPL,N,N
451,6,0605173,TXSFN0605173,HARKEY ROAD,17111 HARKEY ROAD,None,PEARLAND,TX,77584,None,BRAZORIA,48039.0,NaN,NaN,N,Not on the NPL,N,N
452,6,0605177,TXSFN0605177,STATE ROAD 114 GROUNDWATER PLUME,W. STATE ROAD 114,None,LEVELLAND,TX,79336,19,HOCKLEY,48219.0,33.599883,-102.405561,N,Currently on the Final NPL,N,N


In [64]:
# Not needed for final code, seeing types of NPL statuses
npl = sf_sites["NPL"].tolist()
npl_unique = np.unique(npl)
npl_unique

array(['Currently on the Final NPL', 'Deleted from the Final NPL',
       'Not on the NPL', 'Proposed for NPL', 'Site is Part of NPL Site'],
      dtype='<U26')

In [67]:
# Returning new DF contianing sites that have 
sf_sites_cleaned = sf_sites[sf_sites["LATITUDE"].notna()]
sf_sites_cleaned

,RG,SITE_ID,SITE_EPA_ID,SITE_NAME,SITE_STRT_ADRS1,SITE_STRT_ADRS2,SITE_CITY_NAME,SITE_STATE,SITE_ZIP_CODE,SITE_CONG_DISTRICT,SITE_CNTY_NAME,SITE_FIPS_CODE,LATITUDE,LONGITUDE,ALT_AGREEMENT,NPL,FF,NON_NPL_STATUS
2,6,0605264,TX0000605264,BRINE SERVICE COMPANY,NORTHEAST OF INTERSECTION AT IH-37 AND,GOLDSTON ROAD,CORPUS CHRISTI,TX,78409,27,NUECES,48355.0,27.815372,-97.508606,N,Currently on the Final NPL,N,N
5,6,0605329,TX0000605329,PATRICK BAYOU,N 29 43%2755%22/W 95 06%2752.5%22,None,DEER PARK,TX,77536,36,HARRIS,48201.0,29.731944,-95.114583,N,Currently on the Final NPL,N,N
7,6,0605363,TX0000605363,DONNA RESERVOIR AND CANAL SYSTEM,"SOUTH TEXAS, NORTH OF RIO GRANDE RIVER",None,Donna,TX,78537,34,HIDALGO,48215.0,26.112286,-98.086469,N,Currently on the Final NPL,N,N
23,6,0605015,TX0001399435,CITY OF PERRYTON WELL NO. 2,NORTH AMHERST STREET AND SANTA FE AVENUE,None,PERRYTON,TX,79070,13,OCHILTREE,48357.0,36.400556,-100.804722,N,Currently on the Final NPL,N,N
24,6,0605023,TX0001407444,SPRAGUE ROAD GROUND WATER PLUME,WEST 81ST ST AND MARY FRANCIS ST,None,ODESSA,TX,79764,11,ECTOR,48135.0,31.918611,-102.428333,N,Currently on the Final NPL,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,6,0606992,TXN000606992,WEST COUNTY ROAD 112 GROUND WATER,INT. BETWEEN CTY ROAD (CR) 112 %26 CR 1205,None,MIDLAND,TX,79706,11,MIDLAND,48329.0,31.964129,-102.094921,N,Currently on the Final NPL,N,N
346,6,0607093,TXN000607093,US OIL RECOVERY,400 AND 200 N. RICHEY STREET,None,PASADENA,TX,77506,29,HARRIS,48201.0,29.718389,-95.221528,N,Currently on the Final NPL,N,N
391,6,0607441,TXN000607441,MAIN STREET GROUND WATER PLUME,COUNTY ROAD 340,None,BURNET,TX,78611,25,BURNET,48053.0,30.737148,-98.225440,N,Currently on the Final NPL,N,N
396,6,0607499,TXN000607499,NORTHWEST ODESSA GROUNDWATER,Intersection of Andrews Highway and 54th Street,None,ODESSA,TX,79762,None,ECTOR,48135.0,31.894444,-102.388889,N,Proposed for NPL,N,N


In [92]:
lat_long = list(zip(sf_sites_cleaned['LATITUDE'], sf_sites_cleaned['LONGITUDE']))
sf_sites_cleaned['LAT_LONG'] = lat_long
sf_sites_cleaned

,RG,SITE_ID,SITE_EPA_ID,SITE_NAME,SITE_STRT_ADRS1,SITE_STRT_ADRS2,SITE_CITY_NAME,SITE_STATE,SITE_ZIP_CODE,SITE_CONG_DISTRICT,SITE_CNTY_NAME,SITE_FIPS_CODE,LATITUDE,LONGITUDE,ALT_AGREEMENT,NPL,FF,NON_NPL_STATUS,LAT_LONG
2,6,0605264,TX0000605264,BRINE SERVICE COMPANY,NORTHEAST OF INTERSECTION AT IH-37 AND,GOLDSTON ROAD,CORPUS CHRISTI,TX,78409,27,NUECES,48355.0,27.815372,-97.508606,N,Currently on the Final NPL,N,N,"(27.815372, -97.508606)"
5,6,0605329,TX0000605329,PATRICK BAYOU,N 29 43%2755%22/W 95 06%2752.5%22,None,DEER PARK,TX,77536,36,HARRIS,48201.0,29.731944,-95.114583,N,Currently on the Final NPL,N,N,"(29.731944, -95.114583)"
7,6,0605363,TX0000605363,DONNA RESERVOIR AND CANAL SYSTEM,"SOUTH TEXAS, NORTH OF RIO GRANDE RIVER",None,Donna,TX,78537,34,HIDALGO,48215.0,26.112286,-98.086469,N,Currently on the Final NPL,N,N,"(26.112286, -98.086469)"
23,6,0605015,TX0001399435,CITY OF PERRYTON WELL NO. 2,NORTH AMHERST STREET AND SANTA FE AVENUE,None,PERRYTON,TX,79070,13,OCHILTREE,48357.0,36.400556,-100.804722,N,Currently on the Final NPL,N,N,"(36.400556, -100.804722)"
24,6,0605023,TX0001407444,SPRAGUE ROAD GROUND WATER PLUME,WEST 81ST ST AND MARY FRANCIS ST,None,ODESSA,TX,79764,11,ECTOR,48135.0,31.918611,-102.428333,N,Currently on the Final NPL,N,N,"(31.918611, -102.428333)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,6,0606992,TXN000606992,WEST COUNTY ROAD 112 GROUND WATER,INT. BETWEEN CTY ROAD (CR) 112 %26 CR 1205,None,MIDLAND,TX,79706,11,MIDLAND,48329.0,31.964129,-102.094921,N,Currently on the Final NPL,N,N,"(31.964129, -102.094921)"
346,6,0607093,TXN000607093,US OIL RECOVERY,400 AND 200 N. RICHEY STREET,None,PASADENA,TX,77506,29,HARRIS,48201.0,29.718389,-95.221528,N,Currently on the Final NPL,N,N,"(29.718389, -95.221528)"
391,6,0607441,TXN000607441,MAIN STREET GROUND WATER PLUME,COUNTY ROAD 340,None,BURNET,TX,78611,25,BURNET,48053.0,30.737148,-98.225440,N,Currently on the Final NPL,N,N,"(30.737148, -98.22544)"
396,6,0607499,TXN000607499,NORTHWEST ODESSA GROUNDWATER,Intersection of Andrews Highway and 54th Street,None,ODESSA,TX,79762,None,ECTOR,48135.0,31.894444,-102.388889,N,Proposed for NPL,N,N,"(31.894444, -102.388889)"


In [99]:
sf_sites_cleaned.dtypes

RG                      int64
SITE_ID                object
SITE_EPA_ID            object
SITE_NAME              object
SITE_STRT_ADRS1        object
SITE_STRT_ADRS2        object
SITE_CITY_NAME         object
SITE_STATE             object
SITE_ZIP_CODE          object
SITE_CONG_DISTRICT     object
SITE_CNTY_NAME         object
SITE_FIPS_CODE        float64
LATITUDE              float64
LONGITUDE             float64
ALT_AGREEMENT          object
NPL                    object
FF                     object
NON_NPL_STATUS         object
LAT_LONG               object
dtype: object

In [ ]:
def distance_calc(row):
    address_coords = (lat, long)
    coord2 = 
    return distance.distance(address_coords, coord2).miles

In [93]:
sf_sites_cleaned['DISTANCE'] = distance.distance(address_cords, sf_sites_cleaned['LAT_LONG']).miles

ValueError: When creating a Point from sequence, it must not have more than 3 items.

In [ ]:
# Creates DF of locations within [dist] miles of address

dist = 20

# Creates tuple of lat/long from user address
address_cords = (lat, long)

# Iterates through DF to determine which sites are within the specified distance from the user address








coords_2 = (52.406374, 16.9251681)

print(distance.distance(coords_1, coords_2).miles)